# DSPy on Azure OpenAI - 241026

In this notebook, we will first structure a classification problem that is complex enough so that gpt-4o-mini will fail to correctly classify all data points. We will then use DSPy to optimise the prompt and the few-shot examples and show that DSPy optimisation will yield higher accuracy than the default prompt and examples.

The example is to build a legal assistant that classifies legal documents by detecting certain "clauses" in them. Legal Clauses are not fixed set of words. Their wording changes. They can include similar but different words, include punctuation marks etc. (Otherwise keyword search or BM25 would work!)  The documents will be on average 5000 tokens long which will make detection more difficult for gpt-4o-mini.

The code in this notebook has 4 major parts...
1. We will first create the legal documents with gpt-4o.
2. Eval the default prompt and examples with gpt-4o-mini.
3. Use DSPy to optimise the prompt and examples.
4. Eval the optimised prompt and examples.



In [ ]:
# Test AzureOpenAI endpoints 
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

deployment='gpt-4o-mini-eastus-0718'

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
    )
        
# Send a completion call to generate an answer
completion = client.chat.completions.create(
    model=deployment,
    messages = [
        {
        "role": "system",
        "content": "You are an MIT PhD in Physics, specializing in quantum physics."
        },
        {
        "role": "user",
        "content": "What is a black hole?"
        }
    ]
    # max_tokens=4096
)

#print(completion.model_dump_json(indent=2))
content = completion.choices[0].message.content
print(content)
print(len(content))

In [ ]:
#test DSPy on AzureOpenAI
import os 
import dspy
from dotenv import dotenv_values, load_dotenv

load_dotenv()

config=dotenv_values(".env")
azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
embedding_model="text-embedding-3-small-eastus"
deployment='gpt-4o-mini-eastus-0718'



turbo = dspy.AzureOpenAI(
    api_key=api_key,
    api_version= "2024-06-01",
    api_base=azure_endpoint,
    model=deployment,
)

dspy.configure(lm=turbo)


In [ ]:
#Test DSpy with AzureOpenAI 01
qa = dspy.Predict('question: str -> response: str')
qa(question="What is a blackhole?").response

# Create the Contracts 

We will use gpt-4o to create the legal documents.
Each document can have one or more of the 8 clauses we will use to classify the documents. 






In [ ]:
import os
import pandas as pd
import random
import concurrent.futures
from openai import AzureOpenAI
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Define the OpenAI client
deployment = 'gpt-4o-eastus-0806'
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
)

# Create a contracts folder if it doesn't already exist
if not os.path.exists("contracts_advanced"):
    os.makedirs("contracts_advanced")

# Define clause types with descriptions encouraging variation
clauses = [
    ("Non-Disclosure Agreement (NDA) clause", "Ensure confidentiality by limiting information sharing. Vary the length, use synonyms, and include creative wording to reflect different legal styles."),
    ("Termination Clause", "Define conditions for agreement termination, e.g., 'breach of obligations' or 'failure to meet performance standards'. Use varied language and structure."),
    ("Indemnity Clause", "Address liability protection with terms like 'hold harmless' or 'claims arising from third parties'. Use unique phrasing and optional expressions."),
    ("Force Majeure Clause", "Release obligations under unexpected events like natural disasters or government mandates. Include optional wording and different legal terminologies."),
    ("Data Protection Clause", "Define data handling practices to protect privacy, including compliance with GDPR or CCPA. Use diverse expressions and synonyms.")
]

# Ensure balanced number of contracts per class
num_contracts_per_class = 20
total_contracts = num_contracts_per_class * len(clauses)

# Set the average token length for each contract
target_tokens = 3000

# Function to make a single API call with dynamic token control
def generate_clause_variants(clause_type, clause_description, max_tokens=500, max_retries=3):
    messages = [
    {
        "role": "system",
        "content": (
            "You are a highly skilled legal assistant tasked with drafting contract clauses. Each clause should be unique in phrasing, "
            "utilize creative language, and include variations in legal terminology. Focus on making the clause appear as though it could "
            "have been written by different lawyers, incorporating optional phrases and alternative wordings that maintain the clause’s intent."
        )
    },
    {
        "role": "user",
        "content": (
            f"Create a contract clause for '{clause_type}'. Ensure the clause:\n\n"
            f"1. Clearly covers the intended purpose, providing necessary protections or definitions for '{clause_type}'.\n\n"
            f"2. Uses varied wording and synonyms for key terms to enhance uniqueness. Avoid exact repetition of standard legal phrases "
            f"by using alternative ways of expressing core ideas.\n\n"
            f"3. Includes optional phrases and varied sentence structures, making the clause adaptable to different legal interpretations. "
            f"For example, 'one party shall indemnify,' might appear as 'the other party holds no responsibility for claims,' or "
            f"'both parties agree to limit liability in cases...'.\n\n"
            f"4. Is written as if crafted by different legal minds, with subtle style variations—some clauses may be concise and direct, "
            f"others more elaborate, including additional qualifications or conditions where appropriate.\n\n"
            f"5. Avoids excessive specificity or direct keywords, yet includes enough nuanced legal terminology for identification by intent. "
            f"Clause Description: {clause_description}\n\n"
            f"Make it look like it has been written by different lawyers, using optional phrases, synonyms, and creative structure to "
            f"prevent simple keyword detection."
        )
    }
]

    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=deployment,
                messages=messages,
                max_tokens=max_tokens
            )
            return completion.choices[0].message.content.strip()
        
        except Exception as e:
            if "429" in str(e):
                time.sleep(1)  # Basic backoff on rate limit
            else:
                print(f"Error generating clause for '{clause_type}': {e}")
                return ""
    print(f"Failed to generate clause for '{clause_type}' after {max_retries} attempts.")
    return ""

# Pre-generate filler clauses to reuse and avoid extra API calls
filler_clauses = {clause[0]: generate_clause_variants(clause[0], clause[1], max_tokens=400) for clause in clauses}

# Function to generate a complete contract with token-based size control
def generate_and_save_contract(contract_num, clause_type, clause_description, target_tokens=3000):
    contract_sections = []
    labels = {clause[0]: "Absent" for clause in clauses}
    labels[clause_type] = "Present"

    # Track token usage
    current_tokens = 0

    # Generate the main clause for the contract
    clause_text = generate_clause_variants(clause_type, clause_description, max_tokens=600)
    if not clause_text:
        print(f"Warning: Failed to generate text for {clause_type} in contract {contract_num}")
        return None
    contract_sections.append(clause_text)
    current_tokens += len(clause_text.split())

    # Add filler clauses until reaching target token size
    while current_tokens < target_tokens:
        for other_clause, desc in random.sample(clauses, len(clauses)):
            if other_clause != clause_type and current_tokens < target_tokens:
                filler_text = filler_clauses.get(other_clause, "")
                if filler_text:
                    contract_sections.append(filler_text)
                    current_tokens += len(filler_text.split())
    
    # Define the filename with proper numbering
    filename = f"contracts_advanced/contract{contract_num:03}.txt"

    # Write the contract to a file
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(" ".join(contract_sections))
    except Exception as e:
        print(f"Error saving contract {contract_num}: {e}")
        return None

    # Append the labels for DataFrame storage
    return {
        "filename": filename,
        "contract_type": clause_type,
        "contains_nda": labels.get("Non-Disclosure Agreement (NDA) clause", "Absent"),
        "contains_termination": labels.get("Termination Clause", "Absent"),
        "contains_indemnity": labels.get("Indemnity Clause", "Absent"),
        "contains_force_majeure": labels.get("Force Majeure Clause", "Absent"),
        "contains_data_protection": labels.get("Data Protection Clause", "Absent"),
    }

# Function to generate contracts with balanced classes
def generate_balanced_contracts(num_contracts_per_class, target_tokens=3000):
    contract_labels = []
    contract_num = 1
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        futures = []
        
        for clause_type, clause_desc in clauses:
            for _ in range(num_contracts_per_class):
                futures.append(executor.submit(generate_and_save_contract, contract_num, clause_type, clause_desc, target_tokens))
                contract_num += 1
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=total_contracts, desc="Generating contracts"):
            result = future.result()
            if result:
                contract_labels.append(result)

    return contract_labels

# Generate contracts and gather labels
labels = generate_balanced_contracts(num_contracts_per_class, target_tokens=target_tokens)

# Save the labels if contracts were generated
if labels:
    df = pd.DataFrame(labels)
    df.to_csv("contracts_advanced/contract_labels.csv", index=False)
    print("Contract labels saved as contracts_advanced/contract_labels.csv")
else:
    print("No contracts generated.")
#

In [ ]:
import os
import pandas as pd
import random
import concurrent.futures
from openai import AzureOpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import time

# Load environment variables
load_dotenv()

# Initialize OpenAI client with deployment details
deployment = 'gpt-4o-eastus-0806'
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
)

# Ensure directory exists for generated contracts
os.makedirs("contracts_advanced", exist_ok=True)

# Define clause types with varied descriptions for generating diversity
clauses = [
    ("Non-Disclosure Agreement (NDA) clause", "Ensure confidentiality by limiting information sharing. Vary wording, use synonyms, and creative language."),
    ("Termination Clause", "Define conditions for ending the agreement, e.g., breach of obligations or performance failures. Incorporate unique language and structure."),
    ("Indemnity Clause", "Protect against liabilities, using terms like 'hold harmless' or 'third-party claims'. Encourage distinctive phrasing."),
    ("Force Majeure Clause", "Release obligations due to unforeseen events, like natural disasters. Use optional wording and varied legal terms."),
    ("Data Protection Clause", "Outline data handling practices and privacy protection, such as GDPR or CCPA compliance. Use a range of expressions and synonyms.")
]

# Set contract generation parameters
num_contracts_per_class = 20
total_contracts = num_contracts_per_class * len(clauses)
target_tokens = 3000  # Average token length per contract

# Function to generate clause variants with unique legal language
def generate_clause_variants(clause_type, clause_description, max_tokens=500, max_retries=3):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a skilled legal assistant drafting diverse contract clauses. Each clause should be uniquely phrased, "
                "using creative language and variations in legal terminology. Make it appear as though crafted by different lawyers, "
                "with alternative expressions and optional phrasing to prevent pattern detection."
            )
        },
        {
            "role": "user",
            "content": (
                f"Create a contract clause for '{clause_type}' with the following qualities:\n\n"
                f"1. Clearly addresses the purpose, providing necessary protections or definitions.\n"
                f"2. Uses synonyms and varied wording to enhance uniqueness, avoiding standard legal phrases.\n"
                f"3. Includes optional expressions and sentence variations for adaptability.\n"
                f"4. Appears as if written by various legal experts with diverse styles, adding subtle variations.\n"
                f"5. Avoids exact legal terms to reduce keyword detection, while maintaining legal clarity.\n"
                f"Clause Description: {clause_description}"
            )
        }
    ]

    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=deployment,
                messages=messages,
                max_tokens=max_tokens
            )
            return completion.choices[0].message.content.strip()
        except Exception as e:
            if "429" in str(e):
                time.sleep(1)  # Simple backoff for rate limits
            else:
                print(f"Error generating clause for '{clause_type}': {e}")
                return ""
    print(f"Failed to generate clause for '{clause_type}' after {max_retries} attempts.")
    return ""

# Pre-generate filler clauses for reuse to reduce API calls
filler_clauses = {clause[0]: generate_clause_variants(clause[0], clause[1], max_tokens=400) for clause in clauses}

# Function to create a full contract with controlled token length
def generate_and_save_contract(contract_num, clause_type, clause_description, target_tokens=3000):
    contract_sections = []
    labels = {clause[0]: "Absent" for clause in clauses}
    labels[clause_type] = "Present"

    current_tokens = 0

    # Generate primary clause for the contract
    clause_text = generate_clause_variants(clause_type, clause_description, max_tokens=600)
    if not clause_text:
        print(f"Warning: Failed to generate text for {clause_type} in contract {contract_num}")
        return None
    contract_sections.append(clause_text)
    current_tokens += len(clause_text.split())

    # Add filler clauses until reaching the target token count
    while current_tokens < target_tokens:
        for other_clause, _ in random.sample(clauses, len(clauses)):
            if other_clause != clause_type and current_tokens < target_tokens:
                filler_text = filler_clauses.get(other_clause, "")
                if filler_text:
                    contract_sections.append(filler_text)
                    current_tokens += len(filler_text.split())

    # Set filename for contract and save to file
    filename = f"contracts_advanced/contract{contract_num:03}.txt"
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(" ".join(contract_sections))
    except Exception as e:
        print(f"Error saving contract {contract_num}: {e}")
        return None

    # Return label data for DataFrame
    return {
        "filename": filename,
        "contract_type": clause_type,
        "contains_nda": labels.get("Non-Disclosure Agreement (NDA) clause", "Absent"),
        "contains_termination": labels.get("Termination Clause", "Absent"),
        "contains_indemnity": labels.get("Indemnity Clause", "Absent"),
        "contains_force_majeure": labels.get("Force Majeure Clause", "Absent"),
        "contains_data_protection": labels.get("Data Protection Clause", "Absent"),
    }

# Function to generate a balanced set of contracts across clause types
def generate_balanced_contracts(num_contracts_per_class, target_tokens=3000):
    contract_labels = []
    contract_num = 1
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        futures = []
        
        for clause_type, clause_desc in clauses:
            for _ in range(num_contracts_per_class):
                futures.append(executor.submit(generate_and_save_contract, contract_num, clause_type, clause_desc, target_tokens))
                contract_num += 1
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=total_contracts, desc="Generating contracts"):
            result = future.result()
            if result:
                contract_labels.append(result)

    return contract_labels

# Generate contracts and save labels
labels = generate_balanced_contracts(num_contracts_per_class, target_tokens=target_tokens)

# Save label data to CSV if contracts were generated
if labels:
    df = pd.DataFrame(labels)
    df.to_csv("contracts_advanced/contract_labels.csv", index=False)
    print("Contract labels saved as contracts_advanced/contract_labels.csv")
else:
    print("No contracts generated.")


# Eval with gpt-4o-mini & gpt-4o

In [ ]:
import os
import re
import pandas as pd
import random
import concurrent.futures
from openai import AzureOpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import time

# Load environment variables
load_dotenv()

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
)

# Define deployment names for both models
deployments = {
    "gpt-4o-mini": "gpt-4o-mini-eastus-0718",
    "gpt-4o": "gpt-4o-eastus-0806"
}

# Define clause types and map to DataFrame columns
clauses = [
    "Non-Disclosure Agreement (NDA) clause",
    "Termination Clause",
    "Indemnity Clause",
    "Force Majeure Clause",
    "Data Protection Clause"
]

clause_to_column = {
    "Non-Disclosure Agreement (NDA) clause": "contains_nda",
    "Termination Clause": "contains_termination",
    "Indemnity Clause": "contains_indemnity",
    "Force Majeure Clause": "contains_force_majeure",
    "Data Protection Clause": "contains_data_protection"
}

# Load contract label data and standardize column names
df = pd.read_csv("contracts_advanced/contract_labels.csv")
df.columns = [col.lower() for col in df.columns]

# Function to classify contract content for each clause type
def classify_contract(client, deployment, contract_text, clauses, retries=3):
    responses = {clause: "Unknown" for clause in clauses}  # Initialize all responses as "Unknown"
    
    for clause in clauses:
        for attempt in range(retries):
            try:
                completion = client.chat.completions.create(
                    model=deployment,
                    messages=[
                        {"role": "system", "content": "You are a contract classification assistant."},
                        {
                            "role": "user",
                            "content": (
                            f"As a legal analyst, assess whether the following contract contains a clause that fulfills the described purpose. "
                            f"Analyze the contract thoroughly, considering that the clause may be phrased in various ways or use uncommon legal terminology. "
                            f"Focus on the legal intent and function, not just specific terms or phrases. Answer 'Present' or 'Absent'.\n\n"
                            f"Contract: {contract_text}\n\n"
                            f"Your final answer should be 'Present' or 'Absent' on a new line. Assess based on the purpose and function of '{clause}', rather than specific keywords."
                            
                            )
                        }
                    ],
                    max_tokens=50,
                    temperature=0  # Limit response length
                )
                content = completion.choices[0].message.content.strip().lower()
                #responses[clause] = "Present" if "present" in content else "Absent" if "absent" in content else "Unknown"
                match = re.search(r'\b(Present|Absent)\b', content, re.IGNORECASE)
                if match:
                    responses[clause] = match.group(1).capitalize()
                else:
                    responses[clause] = "Unknown"
#                   break
            except Exception as e:
                print(f"Error classifying '{clause}' on attempt {attempt + 1}: {e}")
                time.sleep(1)
    
    return responses

# Function to calculate accuracy for each clause
def calculate_accuracy(predictions, clauses):
    clause_correct_counts = {clause: 0 for clause in clauses}
    clause_total_counts = {clause: 0 for clause in clauses}

    for result in predictions:
        actual_labels = result["actual_labels"]
        predicted_labels = result["predicted_labels"]
        
        for clause in clauses:
            if actual_labels[clause] != "Unknown":
                clause_total_counts[clause] += 1
                if actual_labels[clause].lower() == predicted_labels[clause].lower():
                    clause_correct_counts[clause] += 1

    # Calculate accuracy per clause
    accuracy = {
        clause: (clause_correct_counts[clause] / clause_total_counts[clause] * 100)
        if clause_total_counts[clause] > 0 else 0 for clause in clauses
    }
    return accuracy

# Function to evaluate contracts for each deployment
def evaluate_model(client, deployment, df, clauses):
    predictions = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Evaluating contracts with {deployment}"):
        with open(row["filename"], "r", encoding="utf-8") as file:
            contract_text = file.read()
        
        # Create dictionary of actual labels based on row data
        actual_labels = {
            clause: row.get(clause_to_column[clause], "Unknown") for clause in clauses
        }
        
        # Get predicted labels
        predicted_labels = classify_contract(client, deployment, contract_text, clauses)
        
        # Append results
        predictions.append({
            "filename": row["filename"],
            "actual_labels": actual_labels,
            "predicted_labels": predicted_labels
        })

        # Print intermediary stats every 5 contracts
        if (i + 1) % 5 == 0:
            intermediate_accuracy = calculate_accuracy(predictions, clauses)
            print(f"\nIntermediate Evaluation Results after {i + 1} contracts with {deployment}:")
            for clause, acc in intermediate_accuracy.items():
                print(f"{clause}: {acc:.2f}%")

    return predictions

# Evaluate models and collect results
results = {}
for model_name, deployment_name in deployments.items():
    predictions = evaluate_model(client, deployment_name, df, clauses)
    accuracy = calculate_accuracy(predictions, clauses)
    results[model_name] = accuracy

# Display final results
for model_name, accuracy in results.items():
    print(f"\nFinal Evaluation Results for {model_name}:")
    accuracy_df = pd.DataFrame({
        "Clause": clauses,
        "Accuracy (%)": [accuracy[clause] for clause in clauses]
    })
    print(accuracy_df)


# DSPY Code 

In [22]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import dspy

# Load environment variables
load_dotenv()



# Initialize the Azure OpenAI client in DSPy settings
gpt4 = dspy.AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),
    api_version="2024-06-01",
    api_base=os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT"),
    model="gpt-4o-mini-eastus-0718",
    max_tokens=2000,
    model_type="chat"
)

dspy.settings.configure(lm=gpt4)

# Define the Signature for contract clause classification
class ContractClauseClassification(dspy.Signature):
    """
    As a legal analyst, assess whether the following contract contains clauses that fulfill the described purposes.
    Analyze the contract thoroughly, considering that clauses may be phrased in various ways or use uncommon legal terminology.
    Focus on the legal intent and function, not just specific terms or phrases.
    For each clause type, answer 'Present' or 'Absent'.
    """

    contract_text = dspy.InputField(desc="The full text of the contract to be analyzed.")
    nda_clause = dspy.OutputField(desc="Non-Disclosure Agreement (NDA) clause: Present or Absent")
    termination_clause = dspy.OutputField(desc="Termination Clause: Present or Absent")
    indemnity_clause = dspy.OutputField(desc="Indemnity Clause: Present or Absent")
    force_majeure_clause = dspy.OutputField(desc="Force Majeure Clause: Present or Absent")
    data_protection_clause = dspy.OutputField(desc="Data Protection Clause: Present or Absent")

# Create the ChainOfThought module for contract clause classification
contract_classifier = dspy.ChainOfThought(ContractClauseClassification)

# Load contract label data and standardize column names
df = pd.read_csv("contracts_advanced/contract_labels.csv")
df.columns = [col.lower() for col in df.columns]

# Map clause names to DataFrame columns
clause_to_column = {
    "nda_clause": "contains_nda",
    "termination_clause": "contains_termination",
    "indemnity_clause": "contains_indemnity",
    "force_majeure_clause": "contains_force_majeure",
    "data_protection_clause": "contains_data_protection"
}

# Function to evaluate contracts
def evaluate_contracts(df, contract_classifier):
    predictions = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Evaluating contracts"):
        with open(row["filename"], "r", encoding="utf-8") as file:
            contract_text = file.read()

        # Prepare inputs for the DSPy module
        inputs = {"contract_text": contract_text}

        # Get predictions from the ChainOfThought module
        result = contract_classifier(**inputs)

        # Collect actual labels
        actual_labels = {
            "nda_clause": row.get(clause_to_column["nda_clause"], "Unknown"),
            "termination_clause": row.get(clause_to_column["termination_clause"], "Unknown"),
            "indemnity_clause": row.get(clause_to_column["indemnity_clause"], "Unknown"),
            "force_majeure_clause": row.get(clause_to_column["force_majeure_clause"], "Unknown"),
            "data_protection_clause": row.get(clause_to_column["data_protection_clause"], "Unknown"),
        }

        # Append results
        predictions.append({
            "filename": row["filename"],
            "actual_labels": actual_labels,
            "predicted_labels": {
                "nda_clause": result.nda_clause,
                "termination_clause": result.termination_clause,
                "indemnity_clause": result.indemnity_clause,
                "force_majeure_clause": result.force_majeure_clause,
                "data_protection_clause": result.data_protection_clause,
            },
            "rationale": result.rationale  # Optional: include reasoning
        })

    return predictions

# Function to calculate accuracy for each clause
def calculate_accuracy(predictions):
    clause_correct_counts = {clause: 0 for clause in clause_to_column.keys()}
    clause_total_counts = {clause: 0 for clause in clause_to_column.keys()}

    for result in predictions:
        actual_labels = result["actual_labels"]
        predicted_labels = result["predicted_labels"]
        
        for clause in clause_to_column.keys():
            if actual_labels[clause] != "Unknown":
                clause_total_counts[clause] += 1
                if actual_labels[clause].lower() == predicted_labels[clause].lower():
                    clause_correct_counts[clause] += 1

    # Calculate accuracy per clause
    accuracy = {
        clause: (clause_correct_counts[clause] / clause_total_counts[clause] * 100)
        if clause_total_counts[clause] > 0 else 0 for clause in clause_to_column.keys()
    }
    return accuracy

# Run the evaluation
predictions = evaluate_contracts(df, contract_classifier)

# Calculate accuracy
accuracy = calculate_accuracy(predictions)

# Display results
for clause, acc in accuracy.items():
    print(f"{clause}: {acc:.2f}%")


/Users/ozgurguler/Developer/Projects/dspy/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: AzureOpenAI.__init__() missing 1 required positional argument: 'api_base'